In [ ]:
# !pip install -q transformers

In [1]:
from flask import Flask, render_template, request, redirect, url_for, make_response,jsonify, request
from werkzeug.utils import secure_filename
import time
from datetime import timedelta
from flask_ngrok import run_with_ngrok
import os
import shutil
# import random
# import cv2

In [2]:
import pytesseract
from pytesseract import Output 

pytesseract.pytesseract.tesseract_cmd = 'D:/tesseract-ocr/tesseract.exe'   

In [3]:
from transformers import pipeline # regex==2019.11.1

In [4]:
try:
 from PIL import Image
except ImportError:
 import Image

In [5]:
from transformers import pipeline 

In [6]:
summarization = pipeline("summarization")

In [7]:
def ocr_core(filename):
    """
    This function will handle the core OCR processing of images.
    """ 
    img =Image.open(filename)
    
    img= img.resize((round(img.size[0]*1.5), round(img.size[1]*1.5)))
    text = pytesseract.image_to_string(img)  # We'll use Pillow's Image class to open the image and pytesseract to detect the string in the image
           
    TEXT_TO_SUMMARIZE = text
    summary = summarization(TEXT_TO_SUMMARIZE)
    print(summary[0]['summary_text'])
    summary_output =summary[0]['summary_text']

    return text,summary_output

In [8]:
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'JPG', 'PNG', 'bmp'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS
 
app = Flask(__name__)

# @app.route('/upload', methods=['POST', 'GET'])
@app.route('/upload', methods=['POST', 'GET'])  
def upload():
    if request.method == 'POST':
        f = request.files['file']
 
        if not (f and allowed_file(f.filename)):
            return jsonify({"error": 1001, "msg": "Please Check the format. We allow PNG、jpg、JPG、bmp."})
 
        user_input = request.form.get("name")
 
        basepath = r'C:/Users/Alan He/Desktop/Build/' # os.path.dirname(__file__)  # current directory
        upload_path = os.path.join(basepath, 'static/images', secure_filename(f.filename))  
        f.save(upload_path)
        
        extracted_text, summary_output = ocr_core(f)
        filename1 = f.filename
        print(filename1)
        return render_template('upload_ok.html',  extracted_text=extracted_text , file_name=filename1 , summary_output1 =summary_output ) # ,userinput=user_input,val1=time.time()
 
    return render_template('upload.html')
 
    
app.run(host='0.0.0.0', port=8987, debug=False) # open this website 127.0.0.1:8987/upload

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8987/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Nov/2020 10:03:20] "GET /upload HTTP/1.1" 200 -
